![imagen](../../imagenes/python.jpg)

# SQL en Python
Podrás atacar a una base de datos SQL desde muchas plataformas/lenguajes. Por supuesto, Python es uno de ellos. Para ello **utilizaremos el módulo `sqlite3`**, que viene incluido en la suite de paquetes de Anaconda.


1. [Configuración del entorno](#1.-Configuración-del-entorno)
2. [Modelo de datos](#2.-Modelo-de-datos)
3. [Queries](#3.-Queries)

    3.1 [Primera query](#3.1-Primera-query)
    
    3.2 [SELECT](#3.2-SELECT)
    
    3.3 [LIMIT](#3.3-LIMIT)
    
    3.4 [DISTINCT](#3.4-DISTINCT)
    
    3.5 [WHERE](#3.5-WHERE)
    
    3.6 [ORDER BY](#3.6-ORDER-BY)
    
    3.7 [Agregaciones](#3.7-Agregaciones)
    
    3.8 [GROUP BY](#3.8-GROUP-BY)
    
    3.9 [JOIN](#3.9-JOIN)
    
    3.10 [VIEW y DROP](#3.10-VIEW-y-DROP)
    
4. [Errores](#4.-Errores)

5. [Resumen](#5.-Resumen)


## 1. Configuración del entorno
En este taller usaremos también *Pandas*, que es el módulo para tratamiento de datos tabular que más se usa en Python.

Importamos ambos paquetes

In [95]:
import pandas as pd 
import sqlite3

Lo primero que hacemos es **establecer conexión con la base de datos**. En este taller simplemente leeremos la base de datos de un archivo, pero lo normal es que tengamos que configurar la conexión a una base de datos de la empresa. Para ello existen otras librerías como `pyodbc` en el caso de un SQLServer o `cx_Oracle`, si el servidor es Oracle.

**¡Asegurate que tienes el archivo chinook.db en la ruta donde se encuentra este Notebook!**

In [124]:
conecction = sqlite3.connect("chinook.db")
crsr = conecction.cursor()

In [29]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## 2. Modelo de datos
Antes de empezar a atacar una base de datos, tendremos que saber qué hay dentro, y para ello lo mejor es ver cómo es su **modelo de datos**

![imagen](../../imagenes/chinook_data_model.png)

Puedes ver todas las tablas que hay en la base de datos con la siguiente sentencia

In [30]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track
tracks
sqlite_stat1


## 3. Queries
En las *queries* o consultas, escribiremos nuestra sentencia SQL para poder traernos datos de la base de datos. Las queries tienen esta pinta:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condiciones
```

Por supuesto, hay más sentencias propias de SQL que iremos viendo a lo largo del Notebook. Fíjate que las **palabras reservadas en SQL se suelen poner en mayúsculas**, para diferenciarlas del resto. No da error si se pone de otra manera, ya que SQL **no es *case sensitive***, pero sí se suele hacer así.

### 3.1 Primera query

In [31]:
query = '''
SELECT * 
FROM tracks
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


### 3.2 SELECT
**La sentencia `SELECT` se usa para filtrar columnas**. Podremos incluso renombrar algunas columnas dentro del propio `SELECT`. Dos opciones:
* **SELECT * :** de esta forma lo que haremos es traer todos los campos de la tabla. NO es lo recomendable ya que hay tablas que pueden tener cientos de columnas y probablemente no necesitemos todas, por lo que afectará al rendimiento de la maquina y en definitiva al resto de usuarios. **Recuerda que las bases de datos son normalmente entornos compartidos por varios usuarios :)**
* **SELECT campo1, campo2...:** sentencia para escoger los campos concretos que necesitamos.

Además en la propia query podrás renombrar campos mediante la sintaxis `campo as nuevo_nombre`. **Si quieres poner espacios en el nombre del campo, tendrás que rodear el string con comillas dobles**

SQL no es sensible a mayusculas y minusculas.

In [32]:
query = '''
SELECT *
FROM tracks
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


### 3.3 LIMIT
Se usa para acotar el número de registros de la query. Va siempre al final. Por ejemplo `LIMIT 10`

In [34]:
query = '''
SELECT *
FROM tracks
LIMIT 10

'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
5,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
6,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
7,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
8,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
9,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99


### 3.4 DISTINCT
Se usa para obtener todos los registros únicos, es decir, sin duplicados. Muy útil tanto para eliminar dupicados, como para ver todas las casuísticas de un campo en concreto.

**Mucho cuidado con esta sentencia ya que si la tabla tiene miles o millones de registros, puede ralentizar mucho la query.**

In [33]:
query = '''
SELECT DISTINCT Composer
FROM tracks
'''

sql_query(query)

,Composer
0,"Angus Young, Malcolm Young, Brian Johnson"
1,None
2,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Deaffy & R.A. Smith-Diesel
...,...
848,Carl Nielsen
849,Niccolò Paganini
850,Pietro Antonio Locatelli
851,Claudio Monteverdi


### 3.5 WHERE
Se usa para filtrar filas. Ejemplos de uso:
* **Un valor numérico**
    * UnitPrice = 0.99
    * UnitPrice >= 0.99
    * UnitPrice < 0.99
* **Un valor string**: Name = 'Restless and Wild'
* **Varios valores**: GenreId in (1, 5, 12)
* **string contenido**:
    * strings que empiecen por 'A': Name like 'A%'
    * strings que acaben en 'A': Name like '%A'
    * strings que lleven 'A' en algun punto: Name like '%A%'
* **Distinto de**: UnitPrice <> 0.99

Probamos el `WHERE`

In [35]:
query = '''
SELECT * 
FROM tracks
WHERE unitprice > 0.99
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2819,Battlestar Galactica: The Story So Far,226,3,18,None,2622250,490750393,1.99
1,2820,Occupation / Precipice,227,3,19,None,5286953,1054423946,1.99
2,2821,"Exodus, Pt. 1",227,3,19,None,2621708,475079441,1.99
3,2822,"Exodus, Pt. 2",227,3,19,None,2618000,466820021,1.99
4,2823,Collaborators,227,3,19,None,2626626,483484911,1.99
...,...,...,...,...,...,...,...,...,...
208,3362,"There's No Place Like Home, Pt. 1",261,3,21,None,2609526,522919189,1.99
209,3363,"There's No Place Like Home, Pt. 2",261,3,21,None,2497956,523748920,1.99
210,3364,"There's No Place Like Home, Pt. 3",261,3,21,None,2582957,486161766,1.99
211,3428,Branch Closing,251,3,22,None,1814855,360331351,1.99


Veamos qué tal funciona el `LIKE`

In [41]:
query = '''
SELECT * 
FROM tracks
WHERE composer LIKE "%Brian John%"
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
5,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99
6,11,C.O.D.,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",199836,6566314,0.99
7,12,Breaking The Rules,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,0.99
8,13,Night Of The Long Knives,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205688,6706347,0.99
9,14,Spellbound,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",270863,8817038,0.99


Compliquemos un poco más el `WHERE`

In [43]:
query = '''
SELECT * 
FROM tracks
WHERE unitprice > 0.99 and bytes > 100000000 and genreid in (21,22,23)
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2840,Don't Look Back,228,3,21,None,2571154,493628775,1.99
1,2841,One Giant Leap,228,3,21,None,2607649,521616246,1.99
2,2842,Collision,228,3,21,None,2605480,526182322,1.99
3,2843,Hiros,228,3,21,None,2533575,488835454,1.99
4,2844,Better Halves,228,3,21,None,2573031,549353481,1.99
...,...,...,...,...,...,...,...,...,...
74,3220,Women's Appreciation,251,3,22,None,1732649,338778844,1.99
75,3221,Beach Games,251,3,22,None,1676134,333671149,1.99
76,3222,The Job,251,3,22,None,2541875,501060138,1.99
77,3428,Branch Closing,251,3,22,None,1814855,360331351,1.99


### 3.6 ORDER BY
Podemos **ordenar la tabla por el campo/s que queramos**. Por defecto ordena alfabéticamente los strings y de menor a mayor los tipos numéricos. Si quieres que ordene al revés, tienes que poner `DESC` de la forma `ORDER BY campo DESC`

In [47]:
query = '''
SELECT * 
FROM tracks
WHERE unitprice > 0.99 and bytes > 100000000 and genreid in (21,22,23)
ORDER BY name DESC
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,3220,Women's Appreciation,251,3,22,None,1732649,338778844,1.99
1,2853,Unexpected,228,3,21,None,2598139,511777758,1.99
2,2916,Two for the Road,231,3,21,None,2610958,502404558,1.99
3,2890,Tricia Tanaka Is Dead,229,3,21,None,2635010,548197162,1.99
4,3211,Traveling Salesmen,251,3,22,None,1289039,250822697,1.99
...,...,...,...,...,...,...,...,...,...
74,3213,Ben Franklin,251,3,22,None,1271938,264168080,1.99
75,3221,Beach Games,251,3,22,None,1676134,333671149,1.99
76,3210,Back from Vacation,251,3,22,None,1271688,245378749,1.99
77,3209,"A Benihana Christmas, Pts. 1 & 2",251,3,22,None,2519436,515301752,1.99


In [49]:
query = '''
SELECT * 
FROM tracks
WHERE unitprice > 0.99 and bytes > 100000000 and genreid in (21,22,23)
ORDER BY composer, milliseconds DESC
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,3224,Through a Looking Glass,229,3,21,None,5088838,1059546140,1.99
1,2902,Maternity Leave,231,3,21,None,2780416,555244214,1.99
2,2897,One of Them,231,3,21,None,2698791,542332389,1.99
3,3223,How to Stop an Exploding Man,228,3,21,None,2687103,487881159,1.99
4,2908,One of Us,229,3,21,None,2638096,502387276,1.99
...,...,...,...,...,...,...,...,...,...
74,3213,Ben Franklin,251,3,22,None,1271938,264168080,1.99
75,3210,Back from Vacation,251,3,22,None,1271688,245378749,1.99
76,3214,Phyllis's Wedding,251,3,22,None,1271521,258561054,1.99
77,3218,Safety Training,251,3,22,None,1271229,253054534,1.99


### 3.7 Agregaciones
En ocasiones nos interesa obtener algún estadístico como el máximo de un campo, su desviación estándar o simplemente un conteo de registros no nulos. Para ello podemos usar funciones como `MAX`, `COUNT` o `AVG`. En [esta página](https://www.sqlservertutorial.net/sql-server-aggregate-functions/) encontrarás un resumen con las principales funciones.

In [53]:
query = '''
SELECT COUNT(*)
FROM tracks

'''

sql_query(query)

,COUNT(*)
0,3503


In [54]:
query = '''
SELECT COUNT(composer)
FROM tracks

'''

sql_query(query)

,COUNT(composer)
0,2525


In [55]:
query = '''
SELECT * 
FROM tracks
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [56]:
query = ''' 
SELECT MAX(UnitPrice) as "Precio máximo"
FROM tracks

'''

sql_query(query)

,Precio máximo
0,1.99


### 3.8 GROUP BY
Es una sentencia muy útil para **calcular agregados en función de otro campo**. Por ejemplo, para calcular el total del precio unitario en función del género


In [59]:
query = '''
SELECT genreid, SUM(UnitPrice) as "Tot Price"
FROM tracks
GROUP BY genreid
'''

sql_query(query)

,GenreId,Tot Price
0,1,1284.03
1,2,128.70
2,3,370.26
3,4,328.68
4,5,11.88
5,6,80.19
6,7,573.21
7,8,57.42
8,9,47.52
9,10,42.57


O calcular cuantas canciones hay por compositor

In [64]:
query = '''
SELECT composer, Count(name)
FROM tracks
GROUP BY composer
'''

sql_query(query)

,Composer,Count(name)
0,None,978
1,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",3
2,A. Jamal,1
3,A.Bouchard/J.Bouchard/S.Pearlman,1
4,A.Isbell/A.Jones/O.Redding,1
...,...,...
848,jon lord/roger glover,2
849,lorenz hart/richard rodgers,1
850,orlando murden/ronald miller,1
851,rod mckuen,1


In [78]:
query = '''
SELECT name, composer
FROM tracks
WHERE composer is null
'''

sql_query(query)

,Name,Composer
0,Balls to the Wall,None
1,Desafinado,None
2,Garota De Ipanema,None
3,Samba De Uma Nota Só (One Note Samba),None
4,Por Causa De Você,None
...,...,...
973,Slowness,None
974,"A Midsummer Night's Dream, Op.61 Incidental Mu...",None
975,"Étude 1, In C Major - Preludio (Presto) - Liszt",None
976,"Erlkonig, D.328",None


### 3.9 JOIN
Hasta ahora hemos hecho queries sobre una única tabla, pero **¿y si queremos juntar datos de varias tablas?** Para eso están los `JOIN`s. Para ello **necesitas tener uno o varios campos comunes entre ambas tablas, que denominan CLAVES**.

**¿Cuándo usarlos?** Por ejemplo, si tenemos una tabla con un conjunto de clientes y necesitamos añadirles campos nuevos, tendremos que acudir a otras tablas donde esté ese identificador de cliente y aplicar un `JOIN`. Es lo que se conoce como *pegar campos* a otra tabla.

O imagina que tienes una tabla con todos tus pedidos, con muchos campos(ciudad, dirección, cliente...) y en otra tabla únicamente los números de pedido que no se llegaron a entregar. Si quieres filtrar dentro de tu tabla total de pedidos los que no se llegaron a entregar, podrías aplicar un `INNER JOIN` de manera que te quedes con lo común en ambas tablas, siendo tu clave el identificativo del pedido..

Existen varios tipos de JOINs:

![imagen](../../imagenes/joins.jpg)

In [79]:
query = '''
SELECT genreid
FROM tracks
'''

sql_query(query)

,GenreId
0,1
1,1
2,1
3,1
4,1
...,...
3498,24
3499,24
3500,24
3501,24


In [83]:
query = '''
SELECT * 
FROM genres
'''

sql_query(query)

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


In [88]:
query = '''
SELECT t.name, t.genreid ,g.name
FROM tracks t LEFT OUTER JOIN genres g
ON  t.genreid = g.genreid
ORDER BY t.genreid
'''

sql_query(query)

,Name,GenreId,Name
0,For Those About To Rock (We Salute You),1,Rock
1,Balls to the Wall,1,Rock
2,Fast As a Shark,1,Rock
3,Restless and Wild,1,Rock
4,Princess of the Dawn,1,Rock
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,24,Classical
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",24,Classical
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",24,Classical
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",24,Classical


In [92]:
query = ''' 
SELECT t.trackid, t.name, t.albumid, a.title
FROM tracks t LEFT JOIN albums a
ON a.albumid = t.albumid
'''
sql_query(query)

,TrackId,Name,AlbumId,Title
0,1,For Those About To Rock (We Salute You),1,For Those About To Rock We Salute You
1,2,Balls to the Wall,2,Balls to the Wall
2,3,Fast As a Shark,3,Restless and Wild
3,4,Restless and Wild,3,Restless and Wild
4,5,Princess of the Dawn,3,Restless and Wild
...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,Respighi:Pines of Rome
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,Schubert: The Late String Quartets & String Qu...
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,Monteverdi: L'Orfeo
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,Mozart: Chamber Music


### 3.10 VIEW y DROP
En ocasiones resulta útil **dividir nuestras queries en varios pasos**, y para ello utilizamos la sentencia `VIEW`. Con `VIEW` creamos una *vista* de una tabla, es decir, obtenemos una tabla temporal que usaremos después.

Estas vistas se guardarán en el archivo *.db*, por lo que tendrás que tener cuidado de no almacenar muchas tablas poco útiles.

Primero, veamos cómo acceder a todas las vistas de la base de datos

In [93]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view'")
for name in res:
    print(name[0])

BRIAN


In [112]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view'")
for name in res:
    print(name[0])

BRIAN


Creamos dos vistas nuevas

In [113]:
query1 = ''' 
CREATE VIEW name_composer AS
SELECT name, composer, genreid
FROM tracks
'''

crsr.execute(query1)

In [114]:
query2 = ''' 
CREATE VIEW name_genres AS
SELECT genreid, name
FROM genres 
'''

crsr.execute(query2)

Comprueba que ahora tienes vistas nuevas en tu base de datos

In [115]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view';")
for name in res:
    print(name[0])

BRIAN
name_composer
name_genres


Puedes borrar las vistas mediante `DROP VIEW`. Prueba a borrarlas, comprueba que ya no existen y vuelve a crearlas.

**NOTA**: `DROP` también lo podrás usar como `DROP TABLE`. Nos cargaríamos una tabla entera de la base de datos y ya no sería recuperable, el *ctrl + Z* no te va a salvar. así que mucho cuidado con esta sentencia.

In [111]:
query1 ='''
DROP VIEW IF EXISTS name_composer
'''

query2 ='''
DROP VIEW IF EXISTS name_genres
'''

crsr.execute(query1)
crsr.execute(query2)

Ahora que ya tenemos nuestras vistas, vamos a realizar otra operación con ambas tablas. En este caso, aplicamos un `JOIN`

Fíjate que la salida de esta query es la misma que la del primer `JOIN` de este Notebook, pero en este caso, hecho en dos pasos.

In [117]:
query = '''
SELECT a.name, a.composer, b.name
FROM  name_composer a INNER JOIN name_genres b
ON a.genreid = b.genreid
'''
sql_query(query)

,Name,Composer,Name
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",Rock
1,Balls to the Wall,None,Rock
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Rock
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,Rock
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,None,Classical
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert,Classical
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi,Classical
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart,Classical


## 4. Errores

<table align="left">
 <tr><td width="80"><img src="../../imagenes/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES Acceso a base de datos que no existe</h3>
         
 </td></tr>
</table>

¡Cuidado! No da error, sino que crea una base de datos vacía

In [118]:
conecction = sqlite3.connect("aaa.db")
crsr = conecction.cursor()

<table align="left">
 <tr><td width="80"><img src="../../imagenes/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES en la query</h3>
         
 </td></tr>
</table>

Los errores son todos del mismo tipo: `OperationalError`. Esto nos indica que lo da el propio SQL, no es un error del intérprete de Python como tal. Ahora bien, aunque lo de SQL, devuelve un descriptivo con el tipo de error, que está bastante bien para ver por donde van los tiros.

In [119]:
query = '''
SELEççççççççCT * 
FROM tracks
'''

sql_query(query)

OperationalError: near "SELEççççççççCT": syntax error

In [121]:
query = '''
SELECT * 
FROM tracksssssss
'''

sql_query(query)

OperationalError: no such table: tracksssssss

In [123]:
query = '''
SELECT * 
FROM tracks 
'''

sql_query(query) #Da error porque aunque no hay error de sintaxis sino que la BD se ha creado en vacio... no existe por tanto la tabla

OperationalError: no such table: tracks

In [125]:
conecction = sqlite3.connect("chinook.db")
crsr = conecction.cursor()

In [126]:
query = '''
SELECT campo1, name
FROM tracks 
'''

sql_query(query)

OperationalError: no such column: campo1

## 5. Resumen
**SQL es el estándar que se utiliza en el acceso a las BBDD relacionales**, por lo que en prácticamente cualquier empresa te vas a encontrar una BD de este tipo. La sintaxis es:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condicionales
```

Y podrás realizar las siguientes acciones:
1. **Filtrar columnas**: mediante `SELECT`
2. **Renombrar campos**: mediante `SELECT campo as nuevo_nombre`
3. **Eliminar duplicados**: mediante `DISTINCT`
4. **Limitar el número de registros**: `LIMIT`
5. **Filtrar filas**: `WHERE`
6. **Ordenar una tabla**: `ORDER BY campo1, campo2` y `DESC`
7. **Agregar información**: sacar un KPI como un máximo o un conteo: `MAX`,`COUNT`,`AVG`...
8. **Agregado a nivel grupo**: con `GROUP BY`. Se calculan las funciones de agregación para cada grupo de una columna.
9. **Juntar datos**: mediante `JOIN`. Hay de varios tipos: `LEFT`, `RIGHT`, `FULL`, `INNER` y `OUTER`.
10. **Tablas temporales**: con `VIEW`
11. **Eliminar tablas o vistas**: con `DROP`

Y si te manejas de maravilla con la librería pandas, **siempre puedes obtener tus datos mediante una query con sqlite3, y tratarlos posteriormente en Python, mediante las funciones de pandas.**